In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210725.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,20/07/2021 22:12:55,Three Women (1977),Dead of Night (1945),Dreams (1990),Total Recall (1990),NOT THE BEST PARENTS,[Default to Nicolas Cage],MALE FRONTAL NUDITY,SIGNATURE VEHICLE,NEO-NOIR
1,20/07/2021 22:15:32,Three Women (1977),Dead of Night (1945),Dreams (1990),NaN,NEO-NOIR,MALE FRONTAL NUDITY,SIGNATURE VEHICLE,NOT THE BEST PARENTS,[Default to Nicolas Cage]
2,21/07/2021 05:58:45,Total Recall (1990),Three Women (1977),Dead of Night (1945),NaN,NOT THE BEST PARENTS,NEO-NOIR,SIGNATURE VEHICLE,NaN,NaN
3,21/07/2021 13:22:49,Three Women (1977),Total Recall (1990),Dead of Night (1945),Dreams (1990),NOT THE BEST PARENTS,SIGNATURE VEHICLE,NEO-NOIR,MALE FRONTAL NUDITY,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Three Women (1977)', 'Dead of Night (1945)', 'Dreams (1990)',
       'Total Recall (1990)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Three Women (1977)': <Candidate('Three Women (1977)')>,
 'Dead of Night (1945)': <Candidate('Dead of Night (1945)')>,
 'Dreams (1990)': <Candidate('Dreams (1990)')>,
 'Total Recall (1990)': <Candidate('Total Recall (1990)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Three Women (1977), Dead of Night (1945), Dreams (1990), Total Recall (1990))>,
 <Ballot(Three Women (1977), Dead of Night (1945), Dreams (1990))>,
 <Ballot(Total Recall (1990), Three Women (1977), Dead of Night (1945))>,
 <Ballot(Three Women (1977), Total Recall (1990), Dead of Night (1945), Dreams (1990))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate               Votes  Status
--------------------  -------  --------
Three Women (1977)          3  Elected
Total Recall (1990)         1  Rejected
Dead of Night (1945)        0  Rejected
Dreams (1990)               0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                    Votes  Status
-------------------------  -------  --------
NOT THE BEST PARENTS             3  Elected
NEO-NOIR                         1  Rejected
SIGNATURE VEHICLE                0  Rejected
MALE FRONTAL NUDITY              0  Rejected
[Default to Nicolas Cage]        0  Rejected

